# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,278.96,75,75,12.86,AR,1733008699
1,1,wacheng,33.7833,114.5167,280.02,55,0,3.33,CN,1733008700
2,2,bethel,41.3712,-73.4140,273.65,64,88,0.89,US,1733008702
3,3,yellowknife,62.4560,-114.3525,255.68,78,100,3.60,CA,1733008703
4,4,hotan,37.0997,79.9269,273.05,48,99,1.44,CN,1733008704


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1200,
    frame_height = 1000,
    size = "Humidity",
    scale = 1.5,
    color = "City",
    alpha=0.6,
    hover_cols = ["Country", "Humidity"]
)


# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df.loc[city_data_df["Humidity"]<=60]

# Drop any rows with null values
ideal_cities_clean = ideal_cities.dropna()

# Display sample data
ideal_cities_clean

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,wacheng,33.7833,114.5167,280.02,55,0,3.33,CN,1733008700
4,4,hotan,37.0997,79.9269,273.05,48,99,1.44,CN,1733008704
6,6,hamilton,39.1834,-84.5333,274.31,51,100,4.12,US,1733008700
30,30,hun,29.1268,15.9477,288.72,32,1,2.99,LY,1733008734
36,36,papatowai,-46.5619,169.4708,289.87,43,31,5.61,NZ,1733008741
...,...,...,...,...,...,...,...,...,...,...
554,554,birnin kudu,11.4459,9.4984,291.84,27,79,3.06,NG,1733009705
556,556,pasni,25.2631,63.4710,294.91,58,0,1.57,PK,1733009707
558,558,sarina,-21.4333,149.2167,303.30,1,28,2.27,AU,1733009710
563,563,sinan,34.8262,126.1086,284.85,59,100,3.29,KR,1733009715


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_clean[["City", "Country", "Lat", "Lng", "Humidity"]].copy().reset_index(drop=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,wacheng,CN,33.7833,114.5167,55,
1,hotan,CN,37.0997,79.9269,48,
2,hamilton,US,39.1834,-84.5333,51,
3,hun,LY,29.1268,15.9477,32,
4,papatowai,NZ,-46.5619,169.4708,43,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius=10000
categories="accommodation.hotel"
limit=10

params = {
    "categories":categories,
    "limit":limit, 
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = hotel_df.loc[index,"Lat"]
    longitude = hotel_df.loc[index,"Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wacheng - nearest hotel: No hotel found
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
hamilton - nearest hotel: North Vista Manor
hun - nearest hotel: فندق الهروج
papatowai - nearest hotel: No hotel found
zhob - nearest hotel: No hotel found
tahoua - nearest hotel: Hotel Bungalow
i-n-salah - nearest hotel: باجودة
tobruk - nearest hotel: فندق زهرة البطنان -طبرق
ulladulla - nearest hotel: Hotel Marlin
dunedin - nearest hotel: Scenic Hotel Dunedin City
aioun - nearest hotel: فندق العيون
sitka - nearest hotel: Aspen Suites Hotel
adrar - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
jacobina - nearest hotel: Hotel Jovem Jota
maridi - nearest hotel: No hotel found
gao - nearest hotel: No hotel found
hasaki - nearest hotel: Choshi Plaza Hotel
trevelin - nearest hotel: Casa de Piedra
ras al khaimah city - nearest hotel: Oasis Apartment Hotel
masterton - nearest hotel: Copthorne Hotel Solway Park
goure - nearest

,City,Country,Lat,Lng,Humidity,Hotel Name
0,wacheng,CN,33.7833,114.5167,55,No hotel found
1,hotan,CN,37.0997,79.9269,48,Hua Yi International Hotel (accepts foreigners)
2,hamilton,US,39.1834,-84.5333,51,North Vista Manor
3,hun,LY,29.1268,15.9477,32,فندق الهروج
4,papatowai,NZ,-46.5619,169.4708,43,No hotel found
...,...,...,...,...,...,...
117,birnin kudu,NG,11.4459,9.4984,27,No hotel found
118,pasni,PK,25.2631,63.4710,58,Juddi Hotel
119,sarina,AU,-21.4333,149.2167,1,Tandara Hotel
120,sinan,KR,34.8262,126.1086,59,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1200,
    frame_height = 1000,
    size = 10,
    scale = 3,
    color = "City",
    alpha=0.6,
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)